# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("..\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,14.89,74,0,4.12,US,1707521885
1,1,tromso,69.6496,18.9570,-3.99,58,40,6.69,NO,1707522513
2,2,badger,64.8000,-147.5333,-3.49,77,100,0.00,US,1707522464
3,3,adamstown,-25.0660,-130.1015,25.29,90,100,10.85,PN,1707522448
4,4,tobelo,1.7284,128.0095,27.36,80,68,2.54,ID,1707522513


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]


# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,longboat key,27.4125,-82.6590,21.12,63,0,2.57,US,1707522496
49,49,saint-pierre,-21.3393,55.4781,25.82,73,0,3.09,RE,1707522520
98,98,salalah,17.0151,54.0924,22.05,56,0,2.57,OM,1707522526
158,158,rabigh,22.7986,39.0349,22.50,71,0,0.55,SA,1707522533
196,196,malvan,16.0667,73.4667,23.08,51,0,3.92,IN,1707522538
215,215,margaret river,-33.9500,115.0667,26.49,36,0,2.72,AU,1707522443
216,216,coyhaique,-45.5752,-72.0662,25.34,38,0,3.60,CL,1707522540
273,273,batesville,35.7698,-91.6410,21.23,67,0,3.09,US,1707522550
284,284,south venice,27.0531,-82.4243,21.66,79,0,3.60,US,1707522552
314,314,shahpur,16.7000,76.8333,22.97,70,1,3.86,IN,1707522557


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
40,longboat key,US,27.4125,-82.6590,63,
49,saint-pierre,RE,-21.3393,55.4781,73,
98,salalah,OM,17.0151,54.0924,56,
158,rabigh,SA,22.7986,39.0349,71,
196,malvan,IN,16.0667,73.4667,51,
215,margaret river,AU,-33.9500,115.0667,36,
216,coyhaique,CL,-45.5752,-72.0662,38,
273,batesville,US,35.7698,-91.6410,67,
284,south venice,US,27.0531,-82.4243,79,
314,shahpur,IN,16.7000,76.8333,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
longboat key - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
rabigh - nearest hotel: No hotel found
malvan - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
coyhaique - nearest hotel: No hotel found
batesville - nearest hotel: No hotel found
south venice - nearest hotel: No hotel found
shahpur - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
aldama - nearest hotel: No hotel found
mermoz boabab - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
40,longboat key,US,27.4125,-82.6590,63,No hotel found
49,saint-pierre,RE,-21.3393,55.4781,73,No hotel found
98,salalah,OM,17.0151,54.0924,56,No hotel found
158,rabigh,SA,22.7986,39.0349,71,No hotel found
196,malvan,IN,16.0667,73.4667,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)